In [50]:
# !pip install pandas
# !pip install tensorflow
# !pip install keras
# !pip install sklearn

import pandas as pd
from dataset import get_dataframe

In [51]:
df = get_dataframe('./dataset')

In [52]:
df.head()
list(df)
df.shape

(8475, 28)

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [54]:
# Cleaning dataframe to avoid using NA rating
df.dropna()
dataCleaned = df[df.rating != "NA"]

In [55]:
# Select columns to be used
dataLabel = dataCleaned.rating
dataFrame = dataCleaned[["meta.storyline"]]

In [56]:
dataFrame.head()
dataFrame.shape

(7357, 1)

In [57]:
dataLabel.head()
dataLabel.shape

(7357,)

In [58]:
# Process the labels
labelEncode = LabelEncoder()
Y = labelEncode.fit_transform(dataLabel)
Y = Y.reshape(-1,1)

In [59]:
#print(Y)

In [60]:
# TRAIN AND TEST SPLIT
X_train, X_test, Y_train, Y_test = train_test_split( dataFrame, Y, test_size=0.30 )

In [68]:
# PROCESS DATA :: TOKENIZATION, ADD PADDING
param_max_words = 1000
param_max_len   = 50
tokenizer = Tokenizer( num_words=param_max_words )
tokenizer.fit_on_texts( X_train )
sequences = tokenizer.texts_to_sequences(X_train)
print(sequences)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=param_max_len)
print(sequences_matrix)

[[1, 2]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 2]]


In [69]:
def RNN():
    inputs = Input(name='inputs',shape=[param_max_len])
    layer = Embedding(param_max_words,50,input_length=param_max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model



In [70]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)  

In [72]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

ValueError: Input arrays should have the same number of samples as target arrays. Found 1 input samples and 5149 target samples.

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)


print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

